### [Start Index, End Index] in lap_data correspond to the values time values in heart rate data

In [ ]:
import sql_functions
import get_user_activity_data

# manually getting strava data
access_token = get_user_activity_data.returning_user_access_token('f6cece2afd2ca1d77dc8f397b50cfc2ffe759849') # getting access token
hr_data = get_user_activity_data.get_heart_rate_activity_data(8425197756, access_token)
lap_data = get_user_activity_data.get_activity_laps(8425197756, access_token)
print(hr_data)
print(lap_data)